## Load dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('FakeNews_Data/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [5]:
df.shape

(20800, 5)

In [6]:
# copy df to data
data = df.copy()

In [7]:
data.shape

(20800, 5)

### drop the null values

In [8]:
data = data.dropna()

In [9]:
data.shape

(18285, 5)

### get the independent and dependent features

In [10]:
X = data.drop('label', axis=1)
y = data['label']

In [11]:
print(X.shape)
X.head()

(18285, 4)


,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [12]:
print(y.shape)
y.head()

(18285,)


0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [13]:
### initialize vocab size
vocab_size = 10000

## Onehot Representation

In [14]:
messages = X.copy()

In [15]:
messages.columns

Index(['id', 'title', 'author', 'text'], dtype='object')

In [16]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [17]:
messages.reset_index(inplace=True)

### Dataset Preprocessing using stemming

In [18]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [20]:
len(corpus)

18285

In [21]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding, LSTM, Dense

### Onehot representation of corpus

In [23]:
onehot_repre = [one_hot(words, vocab_size) for words in corpus]
onehot_repre

[[4501, 1418, 107, 5809, 9577, 8912, 7849, 4252, 2665, 8836],
 [2675, 1047, 6355, 6775, 6821, 5236, 788],
 [6535, 3146, 2812, 3948],
 [1735, 5211, 7388, 5406, 4515, 607],
 [3513, 6821, 3934, 5846, 7084, 3337, 6821, 8980, 5428, 4424],
 [4602,
  4238,
  5777,
  4900,
  8474,
  5360,
  2791,
  6277,
  5922,
  3825,
  515,
  8797,
  88,
  503,
  788],
 [3086, 1029, 5094, 9948, 980, 2547, 516, 1845, 7738, 4035, 8779],
 [8899, 7490, 583, 762, 5471, 5481, 5360, 8652, 7738, 4035, 8779],
 [902, 637, 2407, 4995, 5445, 6871, 8690, 3324, 5360, 4375],
 [4764, 9678, 2143, 1863, 3273, 6618, 1489, 1751],
 [686, 7311, 4123, 4668, 9318, 450, 7041, 7618, 65, 5777, 9729],
 [5406, 6778, 9577, 6871, 5360, 5471],
 [1344, 8683, 616, 710, 8500, 8870, 2010, 4317, 5229],
 [9138, 4344, 1259, 7203, 3067, 6320, 4940, 7738, 4035, 8779],
 [5037, 859, 7026, 2394, 3686, 7738, 4035, 8779],
 [9819, 8901, 530, 4439, 6037, 8560, 5864, 9322, 9813, 6976],
 [3601, 8334, 1047],
 [3365, 12, 4896, 1845, 5360, 451, 8151, 788],
 [

### Embedding Representation of onehot encoding

In [24]:
sentence_len = 20
embedded_docs = pad_sequences(onehot_repre, padding='pre', maxlen=sentence_len)
print(embedded_docs)

[[   0    0    0 ... 4252 2665 8836]
 [   0    0    0 ... 6821 5236  788]
 [   0    0    0 ... 3146 2812 3948]
 ...
 [   0    0    0 ... 7738 4035 8779]
 [   0    0    0 ... 9658 1213 6336]
 [   0    0    0 ... 8502  701 1745]]


In [25]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4501,
       1418,  107, 5809, 9577, 8912, 7849, 4252, 2665, 8836])

## Created the Sequential Model

In [26]:
# creating model
dim = 40
model = Sequential()
model.add(Embedding(vocab_size, dim, input_length=sentence_len))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            400000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
print(len(embedded_docs))
print(y.shape)

18285
(18285,)


In [28]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [29]:
print(X_final.shape)
print(y_final.shape)

(18285, 20)
(18285,)


## Model Training

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, train_size=0.8, random_state=42)

In [31]:
# train model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
229/229 [==============================] - 9s 40ms/step - loss: 0.2957 - accuracy: 0.8635 - val_loss: 0.1898 - val_accuracy: 0.9199
Epoch 2/10
229/229 [==============================] - 10s 42ms/step - loss: 0.1149 - accuracy: 0.9569 - val_loss: 0.1972 - val_accuracy: 0.9215
Epoch 3/10
229/229 [==============================] - 9s 37ms/step - loss: 0.0727 - accuracy: 0.9752 - val_loss: 0.2186 - val_accuracy: 0.9207
Epoch 4/10
229/229 [==============================] - 10s 45ms/step - loss: 0.0492 - accuracy: 0.9839 - val_loss: 0.2768 - val_accuracy: 0.9174
Epoch 5/10
229/229 [==============================] - 8s 36ms/step - loss: 0.0366 - accuracy: 0.9891 - val_loss: 0.2939 - val_accuracy: 0.9152
Epoch 6/10
229/229 [==============================] - 11s 48ms/step - loss: 0.0196 - accuracy: 0.9938 - val_loss: 0.3991 - val_accuracy: 0.9122
Epoch 7/10
229/229 [==============================] - 10s 42ms/step - loss: 0.0127 - accuracy: 0.9963 - val_loss: 0.4349 - val_accuracy: 0.

### prediction

In [32]:
y_pred = model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


### Performance Metrics and Accuracy

In [33]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [34]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[1876,  206],
       [ 123, 1452]], dtype=int64)

In [35]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.910035548263604

In [36]:
from sklearn.metrics import classification_report
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.90      0.94      0.92      1999
           1       0.92      0.88      0.90      1658

    accuracy                           0.91      3657
   macro avg       0.91      0.91      0.91      3657
weighted avg       0.91      0.91      0.91      3657

